# debug web
#https://pythononpow.medium.com/debugging-a-tornado-pythononwheels-web-application-with-vscode-d9f5c5e5603f

In [1]:
# {
#     // 使用 IntelliSense 了解相关属性。 
#     // 悬停以查看现有属性的描述。
#     // 欲了解更多信息，请访问: https://go.microsoft.com/fwlink/?linkid=830387
#     "version": "0.2.0",
#     "configurations": [
        
#         {  
#             "name": "Python: PoW",  
#             "type": "python",  
#             "request": "launch",  
#             "program": "${workspaceFolder}/instock/web/web_service.py",  
#             "args": [],  
#             "console": "integratedTerminal"  
#         }
            
#     ]
# }

In [2]:
import logging
import os
import pymysql
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR
from sqlalchemy import inspect
import datetime

import sys

# stdout=sys.stdout
sys.path.append('../../')
from JohnsonUtil import commonTips as cct
from JSONData import tdx_data_Day as tdd

# 初始化db

In [3]:
db_host = "192.168.50.60"  # 数据库服务主机
db_user = "root"  # 数据库访问用户
db_password = "mariadb"  # 数据库访问密码
db_database = "instockdb"  # 数据库名称
db_port = 3306  # 数据库服务端口
db_charset = "utf8mb4"  # 数据库字符集

# 使用环境变量获得数据库,docker -e 传递
_db_host = os.environ.get('db_host')
if _db_host is not None:
    db_host = _db_host
_db_user = os.environ.get('db_user')
if _db_user is not None:
    db_user = _db_user
_db_password = os.environ.get('db_password')
if _db_password is not None:
    db_password = _db_password
_db_database = os.environ.get('db_database')
if _db_database is not None:
    db_database = _db_database
_db_port = os.environ.get('db_port')
if _db_port is not None:
    db_port = int(_db_port)

MYSQL_CONN_URL = "mysql+pymysql://%s:%s@%s:%s/%s?charset=%s" % (
    db_user, db_password, db_host, db_port, db_database, db_charset)

MYSQL_CONN_DBAPI = {'host': db_host, 'user': db_user, 'password': db_password, 'database': db_database,
                    'charset': db_charset, 'port': db_port, 'autocommit': True}



## def func

In [4]:
def write_code_to_blk(codew, blk='060'):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
    if write_blk == 'y' or write_blk == 'Y':
        hdf5_wri = cct.cct_raw_input(
            "Rewrite code [Y] or append [N](defalut:N):")
        if hdf5_wri == 'y' or hdf5_wri == 'Y':
            append_status = False
        else:
            append_status = True
        if len(codew) > 3:
            cct.write_to_blocknew(block_path, codew, append_status,
                                  doubleFile=False, keep_last=0)
            print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write No:%s block_path:%s" % (len(codew), block_path))
            # ("write error:%s block_path:%s" % (len(codew), block_path))
    return True

#allday
def groupby_count_code(df):
    df['couts'] = df.groupby('code')['code'].transform('count')
    if 'date' in df.columns:
        df = df.sort_values(by=['couts','date'],ascending=[0,1])
    else:
        df = df.sort_values(by=['couts'],ascending=[0])
    return df

In [5]:
def get_today(sep='-'):
    TODAY = datetime.date.today()
    fstr = "%Y" + sep + "%m" + sep + "%d"
    today = TODAY.strftime(fstr)
    return today

#前一日
def day_last_days(daynow,last=-1):
    return str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() + datetime.timedelta(last))

#上一个工作日
def last_tddate(days=1):
    # today = datetime.datetime.today().date() + datetime.timedelta(-days)
    if days is None:
        return days
    today = datetime.datetime.today().date()
    # log.debug("today:%s " % (today))
    # return str(today)

    def get_work_day(today):
        day_n = int(today.strftime("%w"))
        if day_n == 0:
            lastd = today + datetime.timedelta(-2)
            # log.debug("0:%s" % lastd)
        elif day_n == 1:
            lastd = today + datetime.timedelta(-3)
            # log.debug("1:%s" % lastd)
        else:
            lastd = today + datetime.timedelta(-1)
            # log.debug("2-6:%s" % lastd)
        return lastd
        # if days==0:
        # return str(lasd)
    lastday = today
    for x in range(int(days)):
        # print x
        lastday = get_work_day(today)
        today = lastday
    return str(lastday)

# 通过数据库链接 engine
def engine():
    return create_engine(MYSQL_CONN_URL)


def engine_to_db(to_db):
    _engine = create_engine(MYSQL_CONN_URL.replace(f'/{db_database}?', f'/{to_db}?'))
    return _engine


# DB Api -数据库连接对象connection，有游标
def conn_with_cursor():
    return conn_not_cursor().cursor()

# DB Api -数据库连接对象connection，无游标
def conn_not_cursor():
    try:
        _db = pymysql.connect(**MYSQL_CONN_DBAPI)
    except Exception as e:
        logging.error(f"database.conn_not_cursor处理异常：{MYSQL_CONN_DBAPI}{e}")
    return _db

# 查询数据
def executeSqlFetch(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.executeSqlFetch处理异常：{sql}{e}")

        result = db.fetchall()
        db.close()
        return result
    
# 计算数量
def executeSqlCount(sql, params=()):
    with conn_with_cursor() as db:
        try:
            db.execute(sql, params)
        except Exception as e:
            logging.error(f"database.select_count计算数量处理异常：{e}")

        result = db.fetchall()
        db.close()
        # 只有一个数组中的第一个数据
        if len(result) == 1:
            return int(result[0][0])
        else:
            return 0

In [6]:
try:
    with pymysql.connect(**MYSQL_CONN_DBAPI) as mydb:
        mydb.cursor().execute(" select 1 ")
except Exception as e:
        logging.error("执行信息：数据库不存在，将创建。")

## 放量上涨table 
cn_stock_strategy_entercn_stock_strategy_enter

In [7]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')


def panda_df(conn,sql):
    df = pd.read_sql(sql, conn)
    print(df.shape)
    return df

from pandas import DataFrame as df
def exe_sql_select(conn,sql):
	cur = conn.cursor() 
	data = None
	try: # 使用异常处理，以防程序无法正常运行
		cur.execute(sql) 
		data = df(cur.fetchall(), columns = [col[0] for col in cur.description]) 
	except Exception as e:
		conn.rollback() # 发生错误时回滚
		print('事务处理失败', e)
	else:
		# conn.commit() # 事务提交
		print('事务处理成功', cur.rowcount)
	# cur.close()
	return data



### 读取SQL数据

In [8]:
conn = pymysql.connect(**MYSQL_CONN_DBAPI)

_selcolall = f''' `date`,`code`,`name`,`rate_1` '''
_selcol = f''' `code`,`name`,`rate_1` '''

_table_name = 'cn_stock_strategy_enter'

lastday = last_tddate(1)

today_now = last_tddate(0)



sql = f'''SELECT '{_selcol}' FROM `{_table_name}` WHERE `date` = '{today_now}' and 
								`kdjk` >= 80 and `kdjd` >= 70 and `kdjj` >= 100 and `rsi_6` >= 80 and 
								`cci` >= 100 and `cr` >= 300 and `wr_6` >= -20 and `vr` >= 160'''

sql_now_price = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' AND rate_1 > 0 '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{lastday}' '''


sql_now_last7day = f'''SELECT {_selcolall} FROM `{_table_name}` WHERE `date` >= '{last_tddate(7)}' '''

sql_now_alltoday = f'''SELECT {_selcolall} FROM `{_table_name}`  '''

sql_tables_name =f'''select table_name from information_schema.tables where table_schema=`{db_database}` '''
# sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where TABLE_SCHEMA=`{db_database}` and table_name = `{_table_name}`'''

sql_tables_name ='SHOW DATABASES'
# print('当前库下所有表的名称')
# sql_columns = "select COLUMN_NAME from information_schema.COLUMNS where table_name = '%s'"%(_table_name)
sql_columns = f'''select COLUMN_NAME from information_schema.COLUMNS where table_name = '{_table_name}' '''
print(sql_now_price)
print(sql_now_today)

mycursor = conn.cursor()
# mycursor.execute(sql_now_price)
# print(_selcol)
# count = 0
# for field in mycursor.fetchall():
# 		# print(field[0],end='  ')
# 		# print(field[1:4])
# 		# print(field)
# 		count +=1
# print("all:%s"%(count))


#lastday 
# df = exe_sql_select(conn,sql_now_price)

#today:
df = exe_sql_select(conn,sql_now_today)

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(conn,sql_now_today)

# df3all = panda_df(conn,sql_now_alltoday)

dflast7d = panda_df(conn,sql_now_last7day)

# mycursor.execute(sql_columns)
# mycursor.execute(sql_now)

# myresult = mycursor.fetchall()
# for x in myresult:
#   print(x)
	
conn.close()

SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-11-16' AND rate_1 > 0 
SELECT  `code`,`name`,`rate_1`  FROM `cn_stock_strategy_enter` WHERE `date` = '2023-11-17' 
事务处理成功 30
(30, 3)
(245, 4)


In [9]:
#today
print(df2[:2],df2.shape)
vol_code = df2.code.tolist()[:2]
#allday
df2[:2]
dflast7d[:2]


     code  name  rate_1
0  000403  派林生物    4.00
1  002119  康强电子    2.01 (30, 3)


,date,code,name,rate_1
0,2023-11-08,002232,启明信息,-3.99
1,2023-11-08,002369,卓翼科技,-4.01


In [10]:
# df3all = groupby_count_code(df3all)
dflast7d = groupby_count_code(dflast7d)

### show couts

In [11]:
filter_date = '2023-04-28'
filter_date = today_now
# last_tddate(0)
# filter_date = lastday

In [12]:
df7multiIndex = dflast7d.reset_index().set_index(['code','date'])
df7multiIndex[:5]
df7tail = df7multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])

### show Today

In [13]:
df7tail =  df7tail.reset_index().set_index('code')
# df7tail[ df7tail.rate_1 ] 
df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())][:6]

,date,index,name,rate_1,couts
code,,,,,
002178,2023-11-17,217,延华智能,10.07,2
603528,2023-11-17,240,多伦科技,10.02,3
603283,2023-11-17,238,赛腾股份,10.00,4
603598,2023-11-17,241,引力传媒,10.00,2
003007,2023-11-17,222,直真科技,10.00,3
002786,2023-11-17,219,银宝山新,9.99,2


In [14]:
df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ][:10]

,date,index,name,rate_1,couts
code,,,,,
002178,2023-11-17,217,延华智能,10.07,2
603528,2023-11-17,240,多伦科技,10.02,3
002983,2023-11-17,221,芯瑞达,10.02,1
603283,2023-11-17,238,赛腾股份,10.00,4
603598,2023-11-17,241,引力传媒,10.00,2
003007,2023-11-17,222,直真科技,10.00,3
002786,2023-11-17,219,银宝山新,9.99,2
600506,2023-11-17,235,统一股份,9.97,2
300765,2023-11-17,231,新诺威,6.72,3


In [15]:
# df7tail =  df7tail.sort_values(by=['rate_1'],ascending=[0])
# df7tail

### 写入放量上涨061

In [16]:
# df7code = df7tail[df7tail.couts >1].index.get_level_values('code')
# filter_date = '2023-04-28' couts >1

df7code = df7tail[(df7tail.couts >1) & (df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date())].index

#nowday
df7code = df7tail[(df7tail.date >= datetime.datetime.strptime(filter_date, '%Y-%m-%d').date()) ].index
df7code
len(df7code)

30

In [17]:

# hdf5_wri = cct.cct_raw_input("rewrite code [Y] or append [N]:")
# if len(hdf5_wri) == 0:
#     pass
# elif hdf5_wri == 'y' or hdf5_wri == 'Y':
#     append_status = False
# else:
#     append_status = True
# df2.code.values
#today:
# write_code_to_blk(df7code,'061')

# write_code_to_blk(df7code,'062')

# write_code_to_blk(df7code,'064')


## 读取cn_stock_strategy_keep_increasing

In [18]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_strategy_keep_increasing'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = '2023-05-08'
start_d = last_tddate(5)
# sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` = '{today_now}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` > '{today_now}'- INTERVAL 5 DAY '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df_increas = panda_df(engine(),sql_now_today)

df_increasA = panda_df(engine(),sql_now_allday)

2023-11-17
(17, 103)
(42, 103)


In [19]:
df_increas = df_increas.sort_values(by=['rate_1'],ascending=[1])
df_increas

,date,code,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,...,rate_91,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100
7,2023-11-17,300063,天龙集团,1.90,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
13,2023-11-17,600360,华微电子,3.89,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
11,2023-11-17,300936,中英科技,3.90,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
9,2023-11-17,300807,天迈科技,3.97,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,2023-11-17,002284,亚太股份,4.23,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
6,2023-11-17,300050,世纪鼎利,5.44,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
8,2023-11-17,300562,乐心医疗,5.60,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
0,2023-11-17,002123,梦网科技,5.72,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
14,2023-11-17,600750,江中药业,6.09,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,2023-11-17,003019,宸展光电,6.71,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## 写入周063

In [20]:
# write_code_to_blk(df_increas.code.tolist(),'063')

### Filter Day多日

In [21]:
df_increas.shape
df_increasA.shape
df_increasA.columns[:10]
# df_iA = df_increasA.groupby(level=[0]).tail(1)
df_iA = groupby_count_code(df_increas)
df_iA_multiIndex = df_iA.reset_index().set_index(['code','date'])
df_iA_multiIndex.shape
# df_iA_multiIndex[:10]
# df_iA_multiIndex[-10:]
# df_iA.loc[df_iA.code == '000028']
df_iA_multiIndex[df_iA_multiIndex.index.get_level_values('code') == '000028'][:2]

,,index,name,rate_1,rate_2,rate_3,rate_4,rate_5,rate_6,rate_7,rate_8,...,rate_92,rate_93,rate_94,rate_95,rate_96,rate_97,rate_98,rate_99,rate_100,couts
code,date,,,,,,,,,,,,,,,,,,,,,


In [22]:
df_iAtail = df_iA_multiIndex.groupby(level=[0]).tail(1)
# df7tail.reset_index().code
df_iAtail =  df_iAtail.reset_index().set_index('code')
df_iAtail.loc[:,['date','name','couts']][:5]
# df_iA.loc[:,['date','code','name','couts']][:50]

# df_iAtail[df_iAtail.couts < 3].loc[:,['date','name','couts']]

,date,name,couts
code,,,
300063,2023-11-17,天龙集团,1
600360,2023-11-17,华微电子,1
300936,2023-11-17,中英科技,1
300807,2023-11-17,天迈科技,1
002284,2023-11-17,亚太股份,1


### show Couts

In [23]:
df_iA.loc[:,['date','code','name','couts']]

,date,code,name,couts
7,2023-11-17,300063,天龙集团,1
13,2023-11-17,600360,华微电子,1
11,2023-11-17,300936,中英科技,1
9,2023-11-17,300807,天迈科技,1
2,2023-11-17,002284,亚太股份,1
6,2023-11-17,300050,世纪鼎利,1
8,2023-11-17,300562,乐心医疗,1
0,2023-11-17,002123,梦网科技,1
14,2023-11-17,600750,江中药业,1
5,2023-11-17,003019,宸展光电,1


## cn_stock_indicators MutilIndex

### 读取SQL数据表

In [24]:
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)

# _selcolall = f''' `date`,`code`,`name`,`rate_1` '''
# _selcol = f''' `code`,`name`,`rate_1` '''
_selcol = f''' *'''
_table_name = 'cn_stock_indicators'
# date = last_tddate(1)
today_now = last_tddate(0)
print(today_now)
# conn = pymysql.connect(**MYSQL_CONN_DBAPI)
start_d = last_tddate(5)

sql_now_today = f'''SELECT {_selcol} FROM `{_table_name}` WHERE `date` >= '{start_d}' '''

#mutildate
sql_now_allday = f'''SELECT {_selcol} FROM `{_table_name}` '''

# mycursor2 = pymysql.connect(**MYSQL_CONN_DBAPI)
df2 = panda_df(engine(),sql_now_today)

# df2 = panda_df(engine(),sql_now_allday)



# conn.close()

2023-11-17
(13532, 77)


In [25]:
df2.columns
df2.loc[df2.code =='000061'][:5]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
43,2023-11-13,000061,农 产 品,6.47,-0.049618,-0.051937,0.002320,26.7034,34.6562,10.7977,...,-0.113514,-0.038817,-47500.0,-65411.8,-69064.0,7.25829,6.60439,5.95049,0.0000,34.1194
4554,2023-11-14,000061,农 产 品,6.50,-0.048264,-0.051308,0.003044,30.8975,33.4033,25.8859,...,-0.101266,-0.079081,128319.0,63742.1,-40865.2,7.25163,6.59833,5.94503,21.9783,20.8112
9064,2023-11-15,000061,农 产 品,6.52,-0.045134,-0.050258,0.005124,38.6539,35.1535,45.6547,...,-0.078635,-0.096573,103218.0,90059.4,-20282.4,7.23387,6.58217,5.93047,36.8771,19.6360


In [26]:
df2[ (df2.close >= df2.boll_ub) & (df2.close * 0.95  <= df2.boll_ub) & (df2.macdh > 0)& (df2.macd > 0)& (df2.macds > 0)]
# .loc[:,['rsi_6','rsi_12','rsi_24','rsi']]

,date,code,name,close,macd,macds,macdh,kdjk,kdjd,kdjj,...,rvi,rvis,fi,force_2,force_13,ene_ue,ene,ene_le,stochrsi_k,stochrsi_d
2,2023-11-13,000004,国华网安,19.52,0.422429,0.151411,0.271018,64.3970,52.3584,88.4742,...,0.295135,0.180828,10665100.0,14371400.0,8406060.0,19.43170,17.68110,15.93050,100.0000,100.0000
33,2023-11-13,000045,深纺织Ａ,13.20,0.555519,0.397623,0.157896,81.7976,81.6720,82.0488,...,0.389191,0.394119,3077750.0,5066920.0,3540960.0,13.54090,12.32100,11.10110,96.4848,84.6775
40,2023-11-13,000058,深 赛 格,7.90,0.149466,0.069606,0.079860,76.1645,76.4611,75.5713,...,0.264516,0.247445,11327900.0,12366700.0,5257840.0,8.16183,7.42653,6.69123,100.0000,88.2922
63,2023-11-13,000158,常山北明,10.93,0.382890,0.376927,0.005963,67.8992,57.7719,88.1538,...,0.206409,0.205767,153641000.0,102093000.0,37147900.0,11.22210,10.21110,9.20010,100.0000,77.4200
226,2023-11-13,000697,*ST炼石,5.86,0.128393,0.006379,0.122013,92.0173,84.6050,106.8420,...,0.188679,0.187753,159796.0,738821.0,527252.0,5.77977,5.25907,4.73837,100.0000,100.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13272,2023-11-15,603722,阿科力,47.64,0.180322,0.111361,0.068961,39.8498,41.0058,37.5378,...,0.087446,0.119825,3489130.0,2311150.0,583439.0,51.61940,46.96900,42.31860,84.8787,41.8055
13293,2023-11-15,603778,乾景园林,5.79,0.112252,0.033016,0.079236,72.4097,59.7320,97.7652,...,0.294118,0.160886,6225590.0,4759740.0,3141840.0,5.93184,5.39744,4.86304,100.0000,95.6537
13387,2023-11-15,603950,长源东谷,16.01,0.578840,0.407202,0.171639,89.4847,89.2591,89.9360,...,0.296619,0.249495,1888700.0,1443450.0,989118.0,16.62220,15.12480,13.62720,92.9946,91.2383
13415,2023-11-15,603995,甬金股份,23.78,0.360394,0.150972,0.209422,78.4157,70.1223,95.0025,...,0.087764,0.086827,38317500.0,25615000.0,5625110.0,24.01930,21.85540,19.69150,100.0000,86.3400


# inStock 序列化数据测试

## 定义参数

In [27]:
stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\2020'+get_today(sep='')[-4:-2]+'\\2020'+get_today(sep='')[-4:]
stock_hist_cache_path

'Y:\\dockerf\\stock\\instockcache\\hist\\202011\\20201117'

In [28]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
get_today()
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\2020'+get_today(sep='')[-4:]
code = '601336'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")


Y:\dockerf\stock\instockcache\hist\202011\20201117\601336qfq.gzip.pickle


In [29]:
df.shape
df[-2:]
# df.columns
lastday=df.date.values[-1]
lastday
data = df.copy()
# macd
data.loc[:, 'diff'], data.loc[:, 'dea'], data.loc[:, 'macd'] = tl.MACD(
    data['close'].values, fastperiod=12, slowperiod=26, signalperiod=9)
data['diff'].values[np.isnan(data['diff'].values)] = 0.0
data['dea'].values[np.isnan(data['dea'].values)] = 0.0
data['macd'].values[np.isnan(data['macd'].values)] = 0.0

data.loc[:, 'boll_ub'], data.loc[:, 'boll'], data.loc[:, 'boll_lb'] = tl.BBANDS \
    (data['close'].values, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
data['boll_ub'].values[np.isnan(data['boll_ub'].values)] = 0.0
data['boll'].values[np.isnan(data['boll'].values)] = 0.0
data['boll_lb'].values[np.isnan(data['boll_lb'].values)] = 0.0
data.iloc[-60:]['macd'].max()
# DIFF:=EMA(CLOSE,12) - EMA(CLOSE,26);  256 
# DEA := EMA(DIFF,9); 1.52
# MACD := 2*(DIFF-DEA); 2.08
print(not False)

(730, 11)

,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
728,2023-11-16,32.52,32.33,32.59,32.18,84322,272621494.0,1.25,-1.16,-0.38,0.40
729,2023-11-17,32.12,32.23,32.26,31.82,77033,246666220.0,1.36,-0.31,-0.10,0.37


'2023-11-17'

0.16855947806231164

True


## stock_period_to_typ Resample 'W-FRI'

In [30]:
def get_tdx_stock_period_to_type(stock_data, period_day='W-FRI', periods=5, ncol=None, ratiodays=True):
    """_周期转换周K,月K_

    Returns:
        _type_: _description_
    """
    #快速日期处理
    #https://www.likecs.com/show-204682607.html

    period_type = period_day
    if 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
    stock_data['date'] = stock_data.index
    lastday = stock_data.date.values[-1]
    lastday2 = stock_data.date.values[-2]
    indextype = True if stock_data.index.dtype == 'datetime64[ns]' else False
    # if cct.get_work_day_status() and 915 < cct.get_now_time_int() < 1500:
    #     stock_data = stock_data[stock_data.index < cct.get_today()]

    if stock_data.index.name == 'date':
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    elif 'date' in stock_data.columns:
        stock_data.set_index('date', inplace=True)
        stock_data.sort_index(ascending=True, inplace=True)
        stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
    # else:
    #     log.error("index.name not date,pls check:%s" % (stock_data[:1]))

    period_stock_data = stock_data.resample(period_type).last()
    # period_stock_data['percent']=stock_data['percent'].resample(period_type,how=lambda x:(x+1.0).prod()-1.0)
    # print stock_data.index[0],stock_data.index[-1]
    # period_stock_data.index =
    # pd.DatetimeIndex(start=stock_data.index.values[0],end=stock_data.index.values[-1],freq='BM')

    period_stock_data['open'] = stock_data[
        'open'].resample(period_type).last()
    period_stock_data['high'] = stock_data[
        'high'].resample(period_type).max()
    period_stock_data['low'] = stock_data[
        'low'].resample(period_type).min()
    
    lastWeek1 = str(period_stock_data['open'].index.values[-1])[:10]
    lastweek2 = str(period_stock_data['open'].index.values[-2])[:10]
    if ratiodays:
        if period_day == 'W-FRI':
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])
            ratio_d =(5-(duratio%5))/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day.find('W') >= 0:
            print(lastWeek1,lastweek2,lastday,lastday2)
            duratio = int(str(datetime.datetime.strptime(lastday, '%Y-%m-%d').date() - datetime.datetime.strptime(lastweek2, '%Y-%m-%d').date())[0])
            ratio_d =(duratio)/5
            print("ratio_d:%s %s"%(ratio_d,lastday))
        elif period_day == 'BM':
            # daynow = '2023-04-26'
            # lastday = '2023-04-23'
            print(lastWeek1,lastweek2,lastday,lastday2)
            # print((str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2]))
            duratio = int(str(datetime.datetime.strptime(lastWeek1, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[:2])
            ratio_d =(30-(duratio%30))/30
            print("ratio_d:%s %s dura:%s"%(ratio_d,lastday,duratio))
        elif period_day.find('M') >= 0:
            ratio_d = 1
            
    else:
        ratio_d = 1
        print(ratio_d)
        
    if ncol is not None:
        for co in ncol:
            period_stock_data[co] = stock_data[co].resample(period_type).sum()
            if ratiodays:
                period_stock_data[co] = period_stock_data[co].apply(lambda x: round(x / ratio_d, 1))
                
    # else:
    period_stock_data['amount'] = stock_data[
        'amount'].resample(period_type).sum()
    period_stock_data['volume'] = stock_data[
        'volume'].resample(period_type).sum()
    if ratiodays and ratio_d != 1:
        period_stock_data['amount'] = period_stock_data['amount'].apply(lambda x: round(x / ratio_d, 1))
        period_stock_data['volume'] = period_stock_data['volume'].apply(lambda x: round(x / ratio_d, 1))
                
    # period_stock_data['turnover']=period_stock_data['vol']/(period_stock_data['traded_market_value'])/period_stock_data['close']
    period_stock_data.index = stock_data['date'].resample(period_type).last().index
    # print period_stock_data.index[:1]
    if 'code' in period_stock_data.columns:
        period_stock_data = period_stock_data[period_stock_data['code'].notnull()]
    period_stock_data = period_stock_data.dropna()
    # period_stock_data.reset_index(inplace=True)
    # period_stock_data.set_index('date',inplace=True)
    # print period_stock_data.columns,period_stock_data.index.name
    # and period_stock_data.index.dtype != 'datetime64[ns]')
    if not indextype and period_stock_data.index.name == 'date':
        # stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d')
        period_stock_data.index = [str(x)[:10] for x in period_stock_data.index]
        period_stock_data.index.name = 'date'
    else:
        if 'date' in period_stock_data.columns:
            period_stock_data = period_stock_data.drop(['date'], axis=1)
    # print period_stock_data
    return period_stock_data


### Resample Week W-FRI

In [31]:
# dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
# dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W-FRI', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='W', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]
# data[-5:]

2023-11-17 2023-11-10 2023-11-17 2023-11-16
ratio_d:1.0 2023-11-17


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-11-03,32.63,32.52,34.22,32.33,660443,2.179241e+09,13.5,-5.5,-0.03,3.2,-1.480377,-1.261268,-0.219108,38.483282,35.3220,32.160718,2023-11-03
2023-11-10,32.50,32.00,33.62,31.80,661207,2.164697e+09,11.3,-1.5,-0.64,3.2,-1.396357,-1.351709,-0.044649,37.841289,34.3955,30.949711,2023-11-10
2023-11-17,32.12,32.23,33.22,31.82,540089,1.746293e+09,8.2,0.7,-0.10,2.6,-1.194234,-1.298635,0.104401,35.597847,33.2805,30.963153,2023-11-17


2023-11-19 2023-11-12 2023-11-17 2023-11-16
ratio_d:1.0 2023-11-17


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-11-05,32.63,32.52,34.22,32.33,660443,2.179241e+09,13.5,-5.5,-0.03,3.2,-1.480377,-1.261268,-0.219108,38.483282,35.3220,32.160718,2023-11-03
2023-11-12,32.50,32.00,33.62,31.80,661207,2.164697e+09,11.3,-1.5,-0.64,3.2,-1.396357,-1.351709,-0.044649,37.841289,34.3955,30.949711,2023-11-10
2023-11-19,32.12,32.23,33.22,31.82,540089,1.746293e+09,8.2,0.7,-0.10,2.6,-1.194234,-1.298635,0.104401,35.597847,33.2805,30.963153,2023-11-17


### Resample Month BM
https://note.nkmk.me/python-pandas-time-series-freq/

    BM: 月末最終営業日ごと
    MS: 毎月の開始日ごと
    BMS: 毎月の最初の営業日ごと


In [32]:
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=False)
dfw[-2:]
dfw = get_tdx_stock_period_to_type(data.copy(), period_day='BM', ncol=['turnover', 'amplitude', 'quote_change'],ratiodays=True)
dfw[-3:]

1


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-10-31,33.25,33.40,38.10,32.33,2110682,7.517695e+09,47.01,-9.53,0.05,10.13,-1.275983,-1.098621,-0.177362,38.396352,35.9780,33.559648,2023-10-31
2023-11-30,32.12,32.23,33.72,31.80,1479480,4.823520e+09,25.53,-3.43,-0.10,7.09,-1.194234,-1.298635,0.104401,35.597847,33.2805,30.963153,2023-11-17


2023-11-30 2023-10-31 2023-11-17 2023-11-16
ratio_d:0.5666666666666667 2023-11-17 dura:13


,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover,diff,dea,macd,boll_ub,boll,boll_lb,date
date,,,,,,,,,,,,,,,,,
2023-09-29,37.13,36.83,43.38,36.73,4462582.9,1.756782e+10,86.5,-15.8,-0.17,21.4,-1.173035,-1.012745,-0.160291,42.824053,39.0720,35.319947,2023-09-28
2023-10-31,33.25,33.40,38.10,32.33,3724732.9,1.326652e+10,83.0,-16.8,0.05,17.9,-1.275983,-1.098621,-0.177362,38.396352,35.9780,33.559648,2023-10-31
2023-11-30,32.12,32.23,33.72,31.80,2610847.1,8.512094e+09,45.1,-6.1,-0.10,12.5,-1.194234,-1.298635,0.104401,35.597847,33.2805,30.963153,2023-11-17


In [33]:
daynow=dfw.index.values[-1]
str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0]
int(str(datetime.datetime.strptime(daynow, '%Y-%m-%d').date() - datetime.datetime.strptime(lastday, '%Y-%m-%d').date())[0])


'1'

1

# inStock RSI

In [34]:
import logging
import os.path
#  import os.sep
import datetime
import numpy as np
import pandas as pd
import talib as tl

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
adjust='qfq'
# stock_hist_cache_path='Y:\\dockerf\\stock\\instockcache\\hist\\202005\\20200530'
code = '000002'
cache_file = os.path.join(stock_hist_cache_path, "%s%s.gzip.pickle" % (code, adjust))
print(cache_file)
df = pd.read_pickle(cache_file, compression="gzip")

Y:\dockerf\stock\instockcache\hist\202011\20201117\000002qfq.gzip.pickle


In [35]:
df.shape
df[-5:]
df[:1]
# df.columns
lastday=df.date.values[-1]
lastday

(730, 11)

,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
725,2023-11-13,11.83,11.72,11.87,11.62,673656,7.887334e+08,2.10,-1.43,-0.17,0.69
726,2023-11-14,11.72,11.78,11.84,11.68,610994,7.179642e+08,1.37,0.51,0.06,0.63
727,2023-11-15,11.99,11.89,12.08,11.88,950347,1.137288e+09,1.70,0.93,0.11,0.98
728,2023-11-16,11.85,11.80,11.89,11.70,622641,7.340102e+08,1.60,-0.76,-0.09,0.64
729,2023-11-17,11.73,11.62,11.80,11.56,639118,7.433935e+08,2.03,-1.53,-0.18,0.66


,date,open,close,high,low,volume,amount,amplitude,quote_change,ups_downs,turnover
0,2020-11-17,26.39,27.04,27.42,26.38,980541,2.936063e+09,3.96,2.85,0.75,1.01


'2023-11-17'

In [36]:
data=df.copy()

In [37]:
def check_rsi_status(df,threshold=60,period_day=False):
    # macd
    data = df.copy()
    # cci 计算方法和结果和stockstats不同，stockstats典型价采用均价(总额/成交量)计算
    data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
    data['cci'].values[np.isnan(data['cci'].values)] = 0.0
    # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
    # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
    rsilast5 = data.iloc[-6:-1]['cci']
    rsilast5max = rsilast5.max()
    rsilast = data.iloc[-1]['cci']

    if rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max:
        return True
    else:
        return False

In [38]:
# data.loc[:, 'cci'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=14)
# data['cci'].values[np.isnan(data['cci'].values)] = 0.0
# # data.loc[:, 'cci_84'] = tl.CCI(data['high'].values, data['low'].values, data['close'].values, timeperiod=84)
# # data['cci_84'].values[np.isnan(data['cci_84'].values)] = 0.0
# rsilast5 = data.iloc[-6:-1]['cci']
# rsilast5max = rsilast5.max()
# rsilast = data.iloc[-1]['cci']
# data['cci'][-6:]
# rsilast5.iloc[-1]
# rsilast5,rsilast
# rsilast5max < 0 and rsilast > rsilast5max and rsilast5.iloc[-1] < rsilast5max
check_rsi_status(df)

False